# WNTR Additional Examples
Here, we will have a few more advanced examples using WNTR focusing on modifying settings and analyzing results:

1. Basic example
2. Basic data processing
3. Loops

TODO: 💡

## Imports
Install and import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Matplotlib is required to create graphics

In [ ]:
# Install required packages if not already available
try:
    import wntr
except ImportError:
    !pip install wntr
    import wntr  # import again after installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Units
WNTR uses **SI (International System) units (length in meters, time in seconds, mass in kilograms)**.  See https://usepa.github.io/WNTR/units.html for more details.

# Basic example

Recall: basics steps:
1. set pat of the .inp file
2. initialize wntr model
3. run simulation
4. get results

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('networks/Net3.inp')

In [ ]:
# Simulate hydraulics using EPANET
sim = wntr.sim.EpanetSimulator(wn)
results_EPANET = sim.run_sim()

Simulation results are stored in an object which includes a dictionary of DataFrames for nodes and a dictionary of DataFrames for links.  Each DataFrame is indexed by time (in seconds) and the columns are node or link names.

In [ ]:
# Print available node results
results_EPANET.node.keys()

In [ ]:
# Print available link results
results_EPANET.link.keys()

In [ ]:
# View EpanetSimulator pressure results
results_EPANET.node['pressure'].head()

In [ ]:
# View EpanetSimulator flow results
results_EPANET.link['flowrate'].head()

In [ ]:
# get pressure at specific node
results_EPANET.node['pressure'].loc[:,'20'] 

In [ ]:
results_pressure = results_EPANET.node['pressure'].loc[:,'20'] 
time_hours = results_pressure.index/3600 
time_hours

In [ ]:
# plot using matplotlib
# pressure junction '20'
# -----------------------------------------------------
results_pressure = results_EPANET.node['pressure'].loc[:,'20'] 
time_hours = results_pressure.index / 3600
plt.plot( time_hours, results_pressure , linewidth=2, alpha=0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Pressure head (m)')
plt.legend('20', title='Junction', loc='best')
plt.savefig('0_pressure.png', dpi = 400)
plt.show()


In [ ]:
results_flow = results_EPANET.link['flowrate'].loc[:,'50'] 

In [ ]:
# plot using matplotlib
# flow rate pipe '50'
# -----------------------------------------------------
results_flow = results_EPANET.link['flowrate'].loc[:,'50'] 
plt.plot( time_hours , results_flow, color = 'red', linewidth = 2, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Flowrate (cms)')
plt.legend(['50'], title='Pipe', loc='best')
plt.savefig('0_flow.png', dpi = 400)
plt.show()

**Let's plot pressures for two nodes on the same plot**

In [ ]:
# pressures at junctions 20 and 107
# -----------------------------------------------------
results_pressure = results_EPANET.node['pressure'].loc[:,'20'] 
plt.plot(time_hours,  results_pressure,  color = 'magenta', linewidth=2, alpha=0.5)

results_pressure = results_EPANET.node['pressure'].loc[:,'107'] 
plt.plot(time_hours, results_pressure,  color = 'green', linewidth=2, alpha=0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Pressure head (m)')
plt.legend(['20','107'], title='Junction', loc='best')
plt.show()

# Basic data processing
In this example, you will:
1. Generate a time-series plot showing  pump flow rate and the tank water level.
2. Plot the tank water level highlighting when it exceeds a chosen threshold and report how many times this occurs.
3. Plot the pump’s ON/OFF status over time and report how many times the pump turns ON.

You will learn how to reference elements in a more general way, which is especially useful for automating simulations and writing adaptable, reusable code.

## Part 1 - plot pump and tank

In [ ]:
# Import .inp and run simulation
inp = 'networks/Net3.inp'
wn = wntr.network.WaterNetworkModel(inp)
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

**Let's start with the pumps**

In [ ]:
# get pump list
print("Pump names", wn.pump_name_list)

In [ ]:
# let's define unit conversion so we can display results in gpm
units = 15850.3 # cms to gpm

In [ ]:
# select pump based on it's ID
pump = wn.get_link('10')
print(pump)    # recall this will print the name of the pump
pump           # recall this will print the variable with all its attributes which is useful when developing the code

In [ ]:
# often times we would like to automate our code and use indexes instead of names (we will see a few examples later)
# this will produce the same result as above but using index instead of name

# select pump index
i = 1
wn.pump_name_list[i]

In [ ]:
pump = wn.get_link(wn.pump_name_list[i])
pump

In [ ]:
# get pump flow 
#---------------------------------------------------------
# note how we are getting pump results by using the variable

results_pump = results.link['flowrate']['335']*units # this gives pump flowrate, but we have to manualy specify pump name

results_pump = results.link['flowrate'][pump.name]*units # this gives pump flowrate but without having to manualy specify the pump

results_pump

In [ ]:
# plot
#---------------------------------------------------------
time_hours = results_pump.index / 3600  # Convert seconds to hours
plt.plot(time_hours, results_pump, color = 'red', linewidth = 2, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Flowrate (gpm)')
plt.legend([pump.name], loc='best')   # this also allows make general and reusable figure legend, titles
plt.title('Pump flow ' + pump.name)
plt.show()

**Let's do the same for tanks**


In [ ]:
# get tank list
print('Tank names', wn.tank_name_list)

In [ ]:
# select a tank
i = 0
tank = wn.get_node(wn.tank_name_list[i])
tank

In [ ]:
# get tank water level and plot

results_tank = results.node['pressure'][tank.name]

plt.plot(time_hours, results_tank, color = 'blue', linewidth = 2, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Tank water level (m)')
plt.legend([tank.name], loc='best') 
plt.title('Tank water level ' + tank.name)

## Part 2 - tank performance
Plot the tank water level highlighting when it exceeds a chosen threshold and report how many times this occurs.

In [ ]:
results_tank = results.node['pressure'][tank.name]
results_tank

In [ ]:
# set threshold
tank_thr = 5.5
# get when tank water level is greater than the threshold
I = results_tank > tank_thr
print(I)

In [ ]:
# get the number of times tank level is above threshold
sum(I)

Python treats `True` and `False` as numbers (`True == 1` and `False == 0`) that's why `sum(I)` gives you the number of times the condition is true

In [ ]:
# print it nicely

print(f'Number of time steps the tank is above {tank_thr} m is = {sum(I)}')

In [ ]:
# plot

plt.plot(time_hours, I, color='blue', linewidth = 2, alpha = 0.5)
plt.title(f'Tank {tank.name} is above {tank_thr} m')
plt.xlabel('Time (hours)')
plt.show()

Change the threshold see that the result makes sense:

higher threshold -> fewer occurances

lower thresshold -> more occrances


## Part 3 - pump performance
Plot the pump’s ON/OFF status over time and report how many times the pump turns ON.

In [ ]:
# get when tank water level is greater than the threshold
I = results_pump > 0 
print(I)

In [ ]:
# print it nicely

print(f'Number of time steps the pump is ON = {sum(I)}')

In [ ]:
print(f'Number of time steps the pump is OFF = {sum(~I)}')

In [ ]:
# OR

print(f'Number of time steps the pump is OFF = {len(I) - sum(I)}')

In [ ]:
# plot

plt.plot(time_hours, I, color='red', linewidth = 2, alpha = 0.5)
plt.title(f'Pump {pump.name} is ON/OFF')
plt.xlabel('Time (hours)')
plt.show()

## Let's plot two results on the same plot

In [ ]:
# 7.5
tank_thr = 💡
# get when tank water level is greater than the threshold
I = results_tank > tank_thr
plt.plot(time_hours, I, color='blue', linewidth = 2, alpha = 0.5)

# 8.5
tank_thr = 💡
# get when tank water level is greater than the threshold
I = results_tank > tank_thr
plt.plot(time_hours, I, color='red', linewidth = 2, alpha = 0.5)
plt.xlabel('Time (hours)')
plt.legend(['7.5','8.5'], loc='best')  
plt.title(f'Tank performance')
plt.show()


# Loops
We often need to loop over multiple elements to modify network components or run stochastic simulations. In this example, we'll loop over all junctions and get the base demand. You can then use similar approach and tailor it to your specific task.

In [ ]:
# first let's revisit a simple loop. you've already seen this in previous lectures
# we can loop using range 

for i in range(8):
        print(i)

In [ ]:
# we can adjust start, and step size
for i in range(20, 7,-2):
        print(i)

Now, let's apply same concept to get demands for multiple junctions. 
* let's recall how to do get demand for a single junction
* then we'll automate by looping over multiple junctions

In [ ]:
# get junction names
node_names = wn.junction_name_list 
print(node_names)

In [ ]:
# get choose specific junction

# let's use junction 15 as an example

i = node_names.index('15') # find the index of '109'
print(i)

In [ ]:
i = 10

temp_node = wn.get_node(node_names[i])
temp_node

In [ ]:
base_demand = temp_node.demand_timeseries_list[0].base_value 
base_demand

In [ ]:
print('Node ID: ' , temp_node.name, '; Base demand = ', base_demand)

In [ ]:
# loop over first three junctions
for i in range (3):
    temp_node = wn.get_node(node_names[i])
    base_demand = temp_node.demand_timeseries_list[0].base_value
    print('Node ID: ' , temp_node, '; Base demand = ', base_demand)

#    for i in range (len(node_names)):

In [ ]:
# loop over all junctions
for i in range (len(node_names)):
    temp_node = wn.get_node(node_names[i])
    base_demand = temp_node.demand_timeseries_list[0].base_value
    print('Node ID: ' , temp_node, '; Base demand = ', base_demand)